In [36]:
from google.cloud import aiplatform
from google.colab import auth
from google.cloud import aiplatform


auth.authenticate_user()
aiplatform.init(project=PROJECT_ID)
import pandas as pd
from tqdm import tqdm
import json
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    GenerationConfig,
    GenerationResponse,
    Tool,
)
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from proto.marshal.collections import repeated
from proto.marshal.collections import maps
from IPython.display import display, Markdown
from google.ai.generativelanguage import Schema,Type
import google.generativeai as genai
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold,
    SafetySetting,
)

from tqdm import tqdm
import re

In [3]:
safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_ONLY_HIGH,
    )
]
model = GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=GenerationConfig(
        temperature=0.00,
    ),
    safety_settings=safety_settings
)

In [29]:
df = pd.read_json('/content/document.json')
df

,source,title,description,content,section
0,https://www.alljitblog.com/%e0%b8%99%e0%b8%ad%...,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,relationship
1,https://www.alljitblog.com/%e0%b8%9e%e0%b8%b4%...,พินัยกรรมแด่ตัวเอง เมื่อความตายอยู่ใกล้ตัวกว่า...,แผนการสำหรับช่วงสุดท้ายของชีวิต หรือ Advance C...,พินัยกรรมแด่ตัวเอง เมื่อความตายอยู่ใกล้ตัวกว่า...,relationship
2,https://www.alljitblog.com/%e0%b9%81%e0%b8%9f%...,แฟนป่วยโรคซึมเศร้า ส่วนเราเอาไงดี? รับมืออย่าง...,,แฟนป่วยโรคซึมเศร้า ส่วนเราเอาไงดี? รับมืออย่าง...,relationship
3,https://www.alljitblog.com/%e0%b9%81%e0%b8%9f%...,แฟนเอาแต่ใจ ทำยังไงดี? ขัดใจเดี๋ยวก็งอนอีก - A...,,แฟนเอาแต่ใจ ทำยังไงดี? ขัดใจเดี๋ยวก็งอนอีก - A...,relationship
4,https://www.alljitblog.com/%e0%b8%a3%e0%b8%b1%...,รักอิ่มตัว หมด Passion มีจริงไหม ในมุมมองนักจิ...,,รักอิ่มตัว หมด Passion มีจริงไหม ในมุมมองนักจิ...,relationship
...,...,...,...,...,...
56,https://www.alljitblog.com/%e0%b8%84%e0%b8%a7%...,ความสัมพันธ์ Toxic คู่ชีวิตที่บั่นทอนจิตใจ - A...,เมื่อความสัมพันธ์ Toxic มาในรูปแบบของ 'ความรัก...,ความสัมพันธ์ Toxic คู่ชีวิตที่บั่นทอนจิตใจ - A...,psycology
57,https://www.alljitblog.com/%e0%b8%84%e0%b8%a7%...,ความเปลี่ยนแปลง เราไม่รู้ว่าต้องเผชิญอะไรบ้าง ...,ความเปลี่ยนแปลง เป็นสิ่งที่หลีกเลี่ยงไม่ได้เรา...,ความเปลี่ยนแปลง เราไม่รู้ว่าต้องเผชิญอะไรบ้าง ...,psycology
58,https://www.alljitblog.com/%e0%b8%84%e0%b8%a7%...,เข้าใจ ความเครียด เครียดแบบไหนควรพบจิตแพทย์ - ...,,เข้าใจ ความเครียด เครียดแบบไหนควรพบจิตแพทย์ - ...,psycology
59,https://www.alljitblog.com/%e0%b9%84%e0%b8%9a%...,ไบโพลาร์ (Bipolar Disorder) อารมณ์ 2 ขั้วโดยจิ...,อย่านำคำว่า ไบโพลาร์ มาใช้ฟุ่มเฟื่อย เพราะคำว่...,ไบโพลาร์ (Bipolar Disorder) อารมณ์ 2 ขั้วโดยจิ...,psycology


In [5]:
 def generate_prompt(title, context):
      prompt = f"""
      <INSTRUCTION>
      <INSTRUCTION>
      - You are tasked with extracting relevant questions based on the provided blog title and the context.
      - The blog title is: {title}
      - The following information is the context taken from the content of the blog:
        Context: {context}

      Your goal is to generate meaningful, specific, and relevant questions that could naturally arise from the provided context.

      Guidelines for question extraction:
      1. Focus on extracting open-ended questions or any questions explicitly or implicitly addressed in the context.
      2. Avoid yes/no questions unless they are crucial to the context.
      3. Ensure the questions cover key aspects, themes, or points mentioned in the context

      </INSTRUCTION>

      """

      example="""Return response in json format: `
      key: title
      value: list of the question
      `

      Sample response: `{
          "เข้าใจ ความเครียด เครียดแบบไหนควรพบจิตแพทย์ ":['เมื่อไหร่ควรหาจิตแพทย์','ความเครียดแบ่งออกเป็นกี่แบบ','จัดการกับความเครียดได้อย่างไร']
      }`

      Response in JSON format:
      """

      prompt=prompt+"\n\n"+example
      return prompt

In [38]:
title=df.iloc[7]['title']
content=df.iloc[7]['content']
section=df.iloc[7]['section']
prompt=generate_prompt(title, content)
response = model.generate_content(
          [prompt],
          generation_config=GenerationConfig(
              temperature=0.0,
              response_mime_type="application/json",
          ),
      )

print(response.text)

{"พฤติกรรมทำลายชีวิตคู่" ในมุมมองนักจิตวิทยา - Alljit Blog": ["นอกเหนือจากความรักแล้ว องค์ประกอบสำคัญอื่นๆ ในการดำเนินชีวิตคู่คืออะไร?", "พฤติกรรมทำลายชีวิตคู่ที่สำคัญที่สุดคืออะไร?", "การไม่จริงใจต่อตัวเองส่งผลต่อความสัมพันธ์อย่างไร?", "การไม่ไว้ใจคู่รักอย่างไร้สาเหตุส่งผลต่อความสัมพันธ์อย่างไร?", "การไม่ซื่อสัตย์ส่งผลต่อความสัมพันธ์อย่างไร?", "การไม่ให้อิสระกับคู่รักส่งผลต่อความสัมพันธ์อย่างไร?", "การพูดไม่คิดหรือพูดตอกย้ำความผิดของคู่รักส่งผลต่อความสัมพันธ์อย่างไร?", "การโกหกส่งผลต่อความสัมพันธ์อย่างไร?", "การคดโกงส่งผลต่อความสัมพันธ์อย่างไร?", "การแสดงพฤติกรรมรุนแรงส่งผลต่อความสัมพันธ์อย่างไร?", "การนอกใจส่งผลต่อความสัมพันธ์อย่างไร?", "การนอกใจและนอกกายแตกต่างกันอย่างไร?", "การนอกใจสามารถแก้ไขได้หรือไม่?", "การนอกกายสามารถแก้ไขได้หรือไม่?", "พฤติกรรมทำลายชีวิตคู่ส่งผลต่อความสัมพันธ์อย่างไร?", "อะไรคือสิ่งที่ควรหลีกเลี่ยงในการดำเนินชีวิตคู่?"]}



In [30]:
ground_truth = pd.DataFrame(columns=['title', 'section', 'question'])

# Iterate through the DataFrame with a progress bar using tqdm
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    title = row['title']
    content = row['content']
    section = row['section']

    # Generate the prompt
    prompt = generate_prompt(title, content)

    # Generate response using the model
    response = model.generate_content(
        [prompt],
        generation_config=GenerationConfig(
            temperature=0.0,
            response_mime_type="application/json",
        ),
    )

    # Try to load JSON response and handle potential exceptions
    try:
        tmp = json.loads(response.text)
    except json.JSONDecodeError:
        # If there's an error in decoding the JSON, skip this iteration
        print(f"Skipping row {index} due to JSONDecodeError")
        continue  # Skip to the next row

    # Append the data to the ground_truth DataFrame
    for key, value in tmp.items():
        for question in value:
            temp_df = pd.DataFrame({'title': [title], 'section': [section], 'question': [question]})
            ground_truth = pd.concat([ground_truth, temp_df], ignore_index=True)

Processing rows:  13%|█▎        | 8/61 [00:11<01:14,  1.40s/it]

Skipping row 7 due to JSONDecodeError


Processing rows:  34%|███▍      | 21/61 [00:55<06:01,  9.05s/it]

Skipping row 20 due to JSONDecodeError


Processing rows:  36%|███▌      | 22/61 [01:22<09:14, 14.21s/it]

...

Processing rows:  57%|█████▋    | 35/61 [02:04<03:47,  8.75s/it]

Skipping row 34 due to JSONDecodeError


Processing rows:  67%|██████▋   | 41/61 [02:35<03:02,  9.14s/it]

Skipping row 40 due to JSONDecodeError


Processing rows: 100%|██████████| 61/61 [02:59<00:00,  2.94s/it]


In [31]:
ground_truth

,title,section,question
0,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,relationship,เมื่อแฟนที่เคยแสนดี นอกใจ มาขอโอกาส เราควรพิจา...
1,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,relationship,การให้โอกาสแฟนที่นอกใจหมายถึงอะไร? มันแตกต่างจ...
2,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,relationship,ทำไมเราถึงรู้สึกอยากให้โอกาสแฟนที่นอกใจ แม้จะร...
3,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,relationship,การให้โอกาสแฟนที่นอกใจอาจนำไปสู่ผลลัพธ์อะไรบ้าง?
4,แฟนที่เคยแสนดี นอกใจ มาขอโอกาส นักจิตวิทยาช่วย...,relationship,เราจะรู้ได้อย่างไรว่าการให้โอกาสแฟนที่นอกใจเป็...
...,...,...,...
544,อยากมี Passion แล้ว Passion ในทางจิตวิทยามีควา...,psycology,เราจะหลีกเลี่ยง Passion ที่ทำร้ายตัวเองได้อย่า...
545,อยากมี Passion แล้ว Passion ในทางจิตวิทยามีควา...,psycology,สาเหตุที่ทำให้เราหมด Passion คืออะไร?
546,อยากมี Passion แล้ว Passion ในทางจิตวิทยามีควา...,psycology,เราจะจัดการกับความรู้สึกหมด Passion ได้อย่างไร?
547,อยากมี Passion แล้ว Passion ในทางจิตวิทยามีควา...,psycology,Passion หายไปหรือแค่เหนื่อย?


In [41]:
ground_truth.to_csv('ground_truth.csv', encoding='utf-8-sig', index=False)